# Calculando a área de π  através de probabilidade

Suponhamos um quadrado de lado 1. Se desenharmos um outro quadrado dentro dele, onde seu lado é 1/2 (portando área 1/4).
Não concorda que, se jogarmos um dardo, a chance de cair dentro do quadrado menor é de 1/4?

Se jogarmos infinitos dardos, a quantidade de dardos dentro do quadrado menor tende a ser 1/4 do número de dardos do quadrado maior.

Vamos fazer esse experimento, mas ao invés de 2 quadrados, vamos desenhar um quadrado e um círculo de **raio 1/2** (diâmetro 1). Ou seja, de área $πr^2$, logo vai receber $πr^2$ do total de dardos.

A formula de uma circunferência é: $$(x-a)²+(y-b)² = r²$$
onde (a,b) é o centro do círculo. Pra nós, o centro do quadrado de lado 1, portanto (1/2, 1/2).

Vamos distribuir infinitos pontos dentro do quadrado e vamos contar quantos caem dentro do círculo.
Para calcular se um ponto está dentro do círculo, checaremos se $$(x-a)²+(y-b)² ≤ r²$$.

Temos que $$\frac{\text{Dardos no Círculo}}{\text{Dardos no Total}} = πr^2⟹π=\frac{\text{Dardos no Círculo}}{\text{Dardos no Total}*r^2}$$

In [9]:
@inline lcg_step(x, a, c, m) = (a * x + c) % m

function generate_points!(points::Array{Tuple{Float64, Float64}}, n)
    resize!(points, n)

    x1 = 274212
    x2 = 679152

    @inbounds for i in 1:n
        # x1 = lcg_step(x1, 6123, 561, 9818)
        # x2 = lcg_step(x2, 6791, 320, 9713)
        x1 = lcg_step(x1, 305641, 30321, 11420107)
        x2 = lcg_step(x2, 279203, 6424214, 9614425)

        points[i] = (x1 / 11420107, x2 / 9614425)
    end

    return points
end

@inline function is_in_circle(x, y)
    dx = x - 0.5f0
    dy = y - 0.5f0
    dx*dx + dy*dy ≤ 0.25f0
end

is_in_circle((x, y)) = is_in_circle(x, y)

is_in_circle (generic function with 2 methods)

In [18]:
using GLMakie

# Criando o círculo.

fig = GLMakie.Figure(size = (800, 800))
axis = GLMakie.Axis(fig[1,1])

colsize!(fig.layout, 1, Aspect(1, 1))

circle = GLMakie.poly!(axis, Circle(Point2f(1/2, 1/2), 1/2), strokewidth = 0.15, color = :transparent, linestyle = :dash)


Poly{Tuple{Circle{Float32}}}

In [19]:
using GLMakie
# Criando o Slider
num_slider = GLMakie.Slider(fig[1,2], range = 0:1000000, horizontal = false, startvalue = 0)
num_points_obs = num_slider.value
|

| (generic function with 30 methods)

In [20]:
using GLMakie
using Makie.ComputePipeline

graph = ComputeGraph()
add_input!(graph, :num_points_obs, num_points_obs)

# Criando os pontos, cores e a fórmula.
map!(graph, [:num_points_obs], [:points_obs, :colors_obs, :formula_label]) do num_points
    # points = Vector{Tuple{Float64, Float64}}(undef, num_points)
    # generate_points!(points, num_points)
    points = [(rand(), rand()) for _ in 0:num_points]
    colors = is_in_circle_map = is_in_circle.(points)
    num_is_in_circle = count(==(1), is_in_circle_map)
    formula_label = L"\frac{%$num_is_in_circle}{%$num_points*0.25} = %$(num_is_in_circle/num_points/0.25)"
    (points, colors, formula_label)
end

points_obs = graph[:points_obs]
colors_obs = graph[:colors_obs]
formula_label = ComputePipeline.get_observable!(graph, :formula_label)

label = Label(fig[2, 1], formula_label, fontsize = 24)

scatter!(axis, points_obs, color = colors_obs, colormap = :redsblues, markersize = 5)
display(fig)

GLMakie.Screen(...)